# Virtual manipulation of datasets tutorial

In this tutorial, we will be using 2 [Dataset types](../../../../doc/#builtin/datasets/Datasets.md.html), namely [Sampled Dataset](../../../../doc/#builtin/datasets/SampledDataset.md.html) and [Transposed Dataset](../../../../doc/#builtin/datasets/TransposedDataset.md.html). We will be using a manually created dataset that has two columns. The first column contains the name of machine learning concepts. The second column contains the description (i.e. first paragraph written about the concept in [Wikipedia](https://en.wikipedia.org)).

The notebook cells below use `pymldb`'s `Connection` class to make [REST API](../../../../doc/#builtin/WorkingWithRest.md.html) calls. You can check out the [Using `pymldb` Tutorial](../../../../doc/nblink.html#_tutorials/Using pymldb Tutorial) for more details.

In [2]:
from pymldb import Connection
mldb = Connection("http://localhost")

## Loading the Dataset
A powerful feature of MLDB allows us to execute SQL code while loading data. Here, the [`tokenize`](../../../../doc/#builtin/sql/ValueExpression.md.html#builtinfunctions) function first splits the columns into (lowercased) individual words, removing words with a length of less than 3 characters. Notice that we have chosen to use a [sparse mutable matrix](../../../../doc/#builtin/datasets/MutableSparseMatrixDataset.md.html) since we do know ahead time the column size for each row.

In [13]:
print mldb.put('/v1/procedures/import_ML_concepts', {
    "type":"import.text",
    "params": {
        "dataFileUrl":"file:///mldb_data/datasets/MachineLearningConcepts.csv",
        "outputDataset": {
                            "id":"ml_concepts", # can be accessed later using id
                            "type": "sparse.mutable" # sparse mutable dataset is needed since we tokenize words below
                         },
        "named": "Concepts", #Row name expression for output dataset
        "select": """     tokenize(
                                lower(Text), 
                                {splitchars: ' -''"?!;:/[]*,().',  
                                min_token_length: 3}) as *
                                
                         """, # Within tokenize function:
                                # lower case
                                # leave out punctuation and spaces
                                # allow only words > 3 characters
        "runOnCreation": True
    }
})

<Response [201]>


## A quick look at the data

We can use the [Query API](../../../../doc/#builtin/sql/QueryAPI.md.html) to get the data into a Pandas DataFrame to take a quick look at it. You will notice that certain cells have a 'NaN' value as seen below. This occurs when the word is not found in the description of the concept. MLDB does uses 'NaN' to more efficiently reprensent a sparse matrix (TBD with François).

In [4]:
mldb.query('SELECT * FROM ml_concepts LIMIT 5')

,addition,algorithm,algorithms,also,analysis,analyze,and,applications,approach,are,...,popularized,provide,rather,recurrent,serve,stored,systems,threshold,understanding,wrong
_rowName,,,,,,,,,,,,,,,,,,,,,
Support vector machine,1,2,1,1,1,1,5,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Logistic regression,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Deep belief network,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Restricted boltzmann machines,NaN,1,2,NaN,NaN,NaN,5,1,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hopfield network,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,...,1,1,1,1,1,1,1,1,1,1


## Exploring the data
Let's count the number of words that describes each concept.

In [5]:
mldb.query("""
SELECT horizontal_sum({*}) as count
FROM ml_concepts

""")

,count
_rowName,
Support vector machine,188
Logistic regression,120
Deep belief network,121
Restricted boltzmann machines,152
Hopfield network,59
Naive bayes classifier,198
Boltzmann machine,112
Autoencoder,41
Artificial neural network,49


## Taking the transpose of data
There are two ways to take the transpose of dataset:
    1. using a transposed dataset
    2. using a transpose SQL function

In [6]:
print mldb.put("/v1/datasets/transposed_concepts", {
    "type": "transposed",
    "params": {
        "dataset": {"id":"ml_concepts"}
    }
})

<Response [201]>


In [7]:
mldb.query("select * from transposed_concepts limit 10")

,Naive bayes classifier,Logistic regression,Hopfield network,Boltzmann machine,Restricted boltzmann machines
_rowName,,,,,
488,1,NaN,NaN,NaN,NaN
718,1,NaN,NaN,NaN,NaN
1950s,1,NaN,NaN,NaN,NaN
1958,NaN,1,NaN,NaN,NaN
1960s,1,NaN,NaN,NaN,NaN
1974,NaN,NaN,1,NaN,NaN
1982,NaN,NaN,1,NaN,NaN
1985,NaN,NaN,NaN,1,NaN
1986,NaN,NaN,NaN,NaN,1


In [8]:
mldb.query("select * from transpose(ml_concepts) limit 10")

,Naive bayes classifier,Logistic regression,Hopfield network,Boltzmann machine,Restricted boltzmann machines
_rowName,,,,,
488,1,NaN,NaN,NaN,NaN
718,1,NaN,NaN,NaN,NaN
1950s,1,NaN,NaN,NaN,NaN
1958,NaN,1,NaN,NaN,NaN
1960s,1,NaN,NaN,NaN,NaN
1974,NaN,NaN,1,NaN,NaN
1982,NaN,NaN,1,NaN,NaN
1985,NaN,NaN,NaN,1,NaN
1986,NaN,NaN,NaN,NaN,1


As you can see, both ways yield the same results.

## Taking a sample of data
There are two ways to take samples of dataset:
    1. using a sampled dataset 
    2. using a sample SQL function

In [9]:
print mldb.put("/v1/datasets/sampled_tokens", {
    "type": "sampled",
    "params": {
        "rows": 20, # enter the number of rows or words to sample randomly
        # "fraction": 0.1, # a fraction of rows can be also be used
        "withReplacement": False, # we don't want to have the same row more than once
        "dataset": {"id":"transposed_concepts"},
        "seed": 0 # Seed value for the random number generator
    }
})

<Response [201]>


In [10]:
mldb.query("select * from sampled_tokens")

,Naive bayes classifier,Artificial neural network,Boltzmann machine,Deep belief network,Logistic regression,Restricted boltzmann machines,Support vector machine,Autoencoder
_rowName,,,,,,,,
maximum,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nervous,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
random,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
such,1,NaN,NaN,1,1,NaN,NaN,NaN
neurons,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
symmetric,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
marked,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
sufficient,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
highly,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
mldb.query("""
            select * 
            from sample(
                        transposed_concepts, 
                        {rows: 20, withReplacement: False})                      
            """)

,Logistic regression,Naive bayes classifier,Autoencoder,Restricted boltzmann machines,Hopfield network,Support vector machine,Boltzmann machine,Artificial neural network,Deep belief network
_rowName,,,,,,,,,
latter,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bayesian,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
more,2,1,1,1,NaN,NaN,NaN,NaN,NaN
understanding,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
called,1,1,NaN,NaN,NaN,2,NaN,NaN,NaN
automatic,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
practical,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
which,1,1,NaN,NaN,NaN,3,NaN,1,NaN
inference,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


As you can see, both methods output a random set of 20 rows.

## Using datasets vs. SQL functions
As seen above, the two different ways to either transpose or sample data are equivalent. It is recommend to use Dataset types instead of SQL functions when the created table will be reused or called later in the program.